In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
await client.get_topics()

In [ ]:
topics = await client.get_topics()
for topic in topics:
    if 'ATCamera' in topic:
        print(topic)

In [ ]:
await client.get_fields('lsst.sal.ATCamera.focal_plane_Reb')

In [ ]:
start = Time("2023-03-20 23:30:00Z", scale='utc')
end = Time("2023-03-21 12:10:00Z", scale='utc')
temps_uncontrolled = await client.select_time_series('lsst.sal.ATCamera.focal_plane_Reb', \
                                            ['temp20', 'private_kafkaStamp'],  start, end)
start = Time("2023-03-21 23:30:00Z", scale='utc')
end = Time("2023-03-22 12:10:00Z", scale='utc')
temps_controlled = await client.select_time_series('lsst.sal.ATCamera.focal_plane_Reb', \
                                            ['temp20', 'private_kafkaStamp'],  start, end)
temps_controlled = temps_controlled.dropna()
temps_uncontrolled = temps_uncontrolled.dropna()

t1_vals = np.array(temps_uncontrolled.values[:,0])
t2_vals = np.array(temps_controlled.values[:,0])
times1 = np.array(temps_uncontrolled.values[:, 1])
times1 -= times1[0]
times2 = np.array(temps_controlled.values[:, 1])
times2 -= times2[0]


In [ ]:
plt.title("AuxTel WREB fan temperature control\n 20:30 CLT - 09:10 CLT")
plt.plot(times2, t2_vals, color='green', label='Controlled - 22Mar23')
plt.plot(times1, t1_vals, color='blue', label='Uncontrolled - 21Mar23')
plt.legend(loc='lower left')
plt.ylabel("WREB Temp2 (C)")
plt.xlabel("Times (sec)")
plt.savefig("/home/craiglagegit/DATA/WREB_Temp_Control_22Mar23")